### Identifica diferenças geométrias e temáticas entre várias ESRI Shapefiles ##

In [ ]:
from gasp.anls.ovlay import check_shape_diff

# Inputs

SHAPES_TO_COMPARE = {
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v11_lsb.shp'       : "cls_int",
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v12_lsb.shp'       : "cls_int",
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v13_lsb_2x2.tif'   : None,
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v13_lsb_5x5.tif'   : None,
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v13_lsb_10x10.tif' : None,
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v14_lsb_2x2.tif'   : None,
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v14_lsb_5x5.tif'   : None,
    r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v14_lsb_10x10.tif' : None
}

OUT_FOLDER = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lsb_anls'
REPORT     = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lsb_compare.xlsx'

conPARAM = {
    "HOST" : "localhost", "PORT" : "5432",
    "USER" : "postgres", "PASSWORD" : "admin", "TEMPLATE" : "postgis_template"
}

DB = "lsb_compare"

srs_code = 3857

RASTER_TEMPLATE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\lisboa\v14_lsb_2x2.tif'

check_shape_diff(SHAPES_TO_COMPARE, OUT_FOLDER, REPORT, conPARAM, DB, srs_code,
                GIS_SOFTWARE="GRASS", RST_GRASS_TEMPLATE=RASTER_TEMPLATE)

### Union Analysis ###

In [ ]:
from gasp.anls.ovlay import union

LYR_A  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v13_coimbra.shp'
LYR_B  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_coimbra_clp.shp'
outSHP = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\teste_v13_v11.shp'
API    = 'saga'

union(LYR_A, LYR_B, outSHP, api_gis=API)

### Union Analysis in GRASS GIS ###

In [ ]:
from gasp.cpu.grs import run_grass
from gasp.oss import get_filename

GRS_WORKSPACE = r'D:\indev\OSM2LULC\WEB_Via\results_fev19'
LYR_A  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v13_coimbra.shp'
LYR_B  = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_coimbra_clp.shp'
outSHP = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\grass_v13_v11.shp'

grsbase = run_grass(GRS_WORKSPACE, location='tst_union', srs=3763)
import grass.script.setup as gsetup
gsetup.init(grsbase, GRS_WORKSPACE, 'tst_union', 'PERMANENT')

# Import data
from gasp.to.shp.grs import shp_to_grs

lyr_a = shp_to_grs(LYR_A, get_filename(LYR_A), asCMD=True)
lyr_b = shp_to_grs(LYR_B, get_filename(LYR_B), asCMD=True)

import datetime
time_a = datetime.datetime.now().replace(microsecond=0)

from gasp.anls.ovlay import union
shpUnion = union(lyr_a, lyr_b, get_filename(outSHP), api_gis="grass_cmd")

time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

# Export data
from gasp.to.shp.grs import grs_to_shp
result = grs_to_shp(shpUnion, outSHP, "area")

In [1]:
"""
Optimized Union Analysis
"""

import os
from gasp.oss import get_filename
from gasp.cpu.gdl.mng.sample import create_fishnet
from gasp.mng.feat import eachfeat_to_newshp
from gasp.cpu.grs import run_grass

LYR_A        = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v11_cmb.shp'
LYR_B        = r'D:\indev\OSM2LULC\WEB_Via\results_fev19\coimbra\v12_cmb.shp'
OUT_FOLDER   = r'D:\indev\OSM2LULC\WEB_Via\tst_union'
REF_BOUNDARY = r'D:\indev\OSM2LULC\WEB_Via\boundaries\coimbra_20x20.shp'
EPSG = 3857

# Create Fishnet
gridShp = create_fishnet(
    REF_BOUNDARY, os.path.join(OUT_FOLDER, 'ref_grid.shp'),
    rowN=4, colN=4
)

# Split Fishnet in several files
cellsShp = eachfeat_to_newshp(gridShp, OUT_FOLDER, epsg=EPSG)

# INIT GRASS GIS Session
grsbase = run_grass(OUT_FOLDER, location="proc", srs=REF_BOUNDARY)
import grass.script.setup as gsetup
gsetup.init(grsbase, OUT_FOLDER, "proc", 'PERMANENT')

import datetime
time_a = datetime.datetime.now().replace(microsecond=0)

# Add data to GRASS GIS
from gasp.to.shp.grs import shp_to_grs
for shp in cellsShp + [LYR_A, LYR_B]:
    shp_to_grs(shp, get_filename(shp), asCMD=True)

# Clip Layers A and B for each CELL in fishnet
from gasp.anls.ovlay import clip
LYRS_A = [clip(
    get_filename(LYR_A), get_filename(cellsShp[x]),
    get_filename(LYR_A) + "_" + str(x), api_gis="grass_cmd"
) for x in range(len(cellsShp))]

time_b = datetime.datetime.now().replace(microsecond=0)

LYRS_B = [clip(
    get_filename(LYR_B), get_filename(cellsShp[x]),
    get_filename(LYR_B) + "_" + str(x), api_gis="grass_cmd"
) for x in range(len(cellsShp))]

time_c = datetime.datetime.now().replace(microsecond=0)

print("Total Time:")
print(time_c - time_a)
print("LYR_A_CLIP")
print(time_b - time_a)
print("LYR_B_CLIP")
print(time_c - time_b)

ValueError: Output: b''
Error: b'Traceback (most recent call last):\r\n  File "C:\\Program Files\\GRASS GIS 7.6.0/scripts/v.clip.py", line 198, in <module>\r\n    opt, flg = parser()\r\n  File "C:\\Program Files\\GRASS GIS 7.6.0\\etc\\python\\grass\\script\\core.py", line 781, in parser\r\n    environ[b\'CMDLINE\'] = b\' \'.join(cmdline)\r\n  File "C:\\Python37\\lib\\os.py", line 682, in __setitem__\r\n    key = self.encodekey(key)\r\n  File "C:\\Python37\\lib\\os.py", line 744, in encodekey\r\n    return encode(key).upper()\r\n  File "C:\\Python37\\lib\\os.py", line 739, in check_str\r\n    raise TypeError("str expected, not %s" % type(value).__name__)\r\nTypeError: str expected, not bytes\r\n'